In [ ]:
!pip install mord
!pip install dmba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mord: filename=mord-0.6-py3-none-any.whl size=6006 sha256=c6ca21c8a2e64454f8e8078bb00d27ee2a27ce5c291a5dcb4d128eb1410a5b33
  Stored in directory: /root/.cache/pip/wheels/80/1d/8a/bef1a01b6a3c91494905d161d5426c181819f9730a773141eb
Successfully built mord
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 2.4 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from dmba import adjusted_r2_score, AIC_score, BIC_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#since there are too many classes, we may treat the problem as a numerical outcome prediction problem 
#and apply linear regression over it

In [ ]:
x_train=pd.DataFrame(np.load('X_train.npy'))
x_train.shape

(1663, 768)

In [ ]:
x_test=pd.DataFrame(np.load('X_test.npy'))

In [ ]:
y_train=pd.read_csv('y_train.csv')

In [ ]:
y_train.drop(columns='Id',inplace=True)

In [ ]:
x_train.var() 
#variances in the same range, no need of standardizations

0      0.000871
1      0.000903
2      0.000896
3      0.001287
4      0.001049
         ...   
763    0.001068
764    0.000759
765    0.000937
766    0.000951
767    0.001036
Length: 768, dtype: float32

## **Linear Regression**

Dimension reduction using PCA, will help save computing time

In [ ]:
pca=PCA(n_components=250)

In [ ]:
pca.fit(x_train)

PCA(n_components=250)

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

array([0.05475494, 0.09107381, 0.12226923, 0.15184842, 0.17607905,
       0.19890065, 0.22003889, 0.24040374, 0.2589863 , 0.27685457,
       0.29433876, 0.31039208, 0.3262475 , 0.34158716, 0.35655236,
       0.37015647, 0.3827712 , 0.39513862, 0.40728545, 0.4189422 ,
       0.43021733, 0.4412626 , 0.45199186, 0.46239695, 0.4724035 ,
       0.48211953, 0.49148756, 0.5007123 , 0.5096055 , 0.5183848 ,
       0.526785  , 0.5350509 , 0.5431321 , 0.5509201 , 0.55848914,
       0.5659731 , 0.57334983, 0.58065975, 0.5877889 , 0.5947421 ,
       0.6014746 , 0.60813624, 0.6146683 , 0.6210592 , 0.6273205 ,
       0.6335089 , 0.6395609 , 0.64543974, 0.65115565, 0.65675306,
       0.66228354, 0.66766286, 0.67301637, 0.6782242 , 0.68332374,
       0.6883797 , 0.6933442 , 0.69827884, 0.70307976, 0.7078641 ,
       0.71245724, 0.7170414 , 0.7215198 , 0.7258744 , 0.7301824 ,
       0.7344263 , 0.7385508 , 0.7426265 , 0.74662364, 0.7505986 ,
       0.7544942 , 0.7582902 , 0.7620599 , 0.7658117 , 0.76946

In [ ]:
#we are able to retain 98.16% of variation with 250 components
#as per my observation, there is no significant improvement in variation retention with further increase in # components
#now, i'm going to use these components as my columns

In [ ]:
pca_xtrain=pd.DataFrame(pca.fit_transform(x_train))
#pca_xtrain.head()

In [ ]:
pca_xtest=pd.DataFrame(pca.transform(x_test))
#pca_xtest.head()

In [ ]:
lin_reg=LinearRegression()

In [ ]:
lin_reg.fit(pca_xtrain,y_train)

LinearRegression()

In [ ]:
test_pred=pd.DataFrame(lin_reg.predict(pca_xtest),columns=['Predictions'])

In [ ]:
test_pred['Predictions']=round(test_pred['Predictions'])
test_pred.shape

(713, 1)

In [ ]:
adjusted_r2_score(y_train,lin_reg.predict(pca_xtrain),lin_reg)
#low r2 score

0.5036068899808435

In [ ]:
lin_reg_pred=pd.DataFrame(lin_reg.predict(pca_xtest),columns=['predictions_lin'])

In [ ]:
lin_reg_pred['grouped_class']=np.NaN

In [ ]:
#grouping the classes
i=0
while i<lin_reg_pred.shape[0]:
  if lin_reg_pred['predictions_lin'].loc[i]>=0 and lin_reg_pred['predictions_lin'].loc[i]<6:
    lin_reg_pred['grouped_class'].loc[i]='0-5'
  elif lin_reg_pred['predictions_lin'].loc[i]>=6 and lin_reg_pred['predictions_lin'].loc[i]<11:
    lin_reg_pred['grouped_class'].loc[i]='6-10'
  elif lin_reg_pred['predictions_lin'].loc[i]>=11 and lin_reg_pred['predictions_lin'].loc[i]<16:
    lin_reg_pred['grouped_class'].loc[i]='11-15'
  elif lin_reg_pred['predictions_lin'].loc[i]>=16 and lin_reg_pred['predictions_lin'].loc[i]<21:
    lin_reg_pred['grouped_class'].loc[i]='16-20'
  elif lin_reg_pred['predictions_lin'].loc[i]>=21 and lin_reg_pred['predictions_lin'].loc[i]<26:
    lin_reg_pred['grouped_class'].loc[i]='21-25'
  elif lin_reg_pred['predictions_lin'].loc[i]>=26 and lin_reg_pred['predictions_lin'].loc[i]<31:
    lin_reg_pred['grouped_class'].loc[i]='26-30'
  elif lin_reg_pred['predictions_lin'].loc[i]>=31 and lin_reg_pred['predictions_lin'].loc[i]<36:
    lin_reg_pred['grouped_class'].loc[i]='31-35' 
  else:
    lin_reg_pred['grouped_class'].loc[i]='36-40'
  i+=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
mean_absolute_error(y_train['Predicted'],lin_reg.predict(pca_xtrain))

7.1353714111979585

In [ ]:
lin_reg_pred.to_csv('Lin_Reg_Predictions.csv')

Grouping the classes to ease classification

In [ ]:
y_train['grouped_classes']=np.NaN

In [ ]:
#grouping the classes
i=0
while i<y_train.shape[0]:
  if y_train['Predicted'].loc[i]>=0 and y_train['Predicted'].loc[i]<6:
    y_train['grouped_classes'].loc[i]='0-5'
  elif y_train['Predicted'].loc[i]>=6 and y_train['Predicted'].loc[i]<11:
    y_train['grouped_classes'].loc[i]='6-10'
  elif y_train['Predicted'].loc[i]>=11 and y_train['Predicted'].loc[i]<16:
    y_train['grouped_classes'].loc[i]='11-15'
  elif y_train['Predicted'].loc[i]>=16 and y_train['Predicted'].loc[i]<21:
    y_train['grouped_classes'].loc[i]='16-20'
  elif y_train['Predicted'].loc[i]>=21 and y_train['Predicted'].loc[i]<26:
    y_train['grouped_classes'].loc[i]='21-25'
  elif y_train['Predicted'].loc[i]>=26 and y_train['Predicted'].loc[i]<31:
    y_train['grouped_classes'].loc[i]='26-30'
  elif y_train['Predicted'].loc[i]>=31 and y_train['Predicted'].loc[i]<36:
    y_train['grouped_classes'].loc[i]='31-35' 
  else:
    y_train['grouped_classes'].loc[i]='36-40'
  i+=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Logistic regression for classification

In [ ]:
logistic_reg = LogisticRegression(penalty="l2", C=1e42, solver='lbfgs')
logistic_reg.fit(pca_xtrain,y_train['grouped_classes'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=1e+42)

In [ ]:
Predictions=pd.DataFrame(logistic_reg.predict(pca_xtest),columns=['Predictions'])
Predictions.shape

(713, 1)

In [ ]:
y_train['grouped_classes'].unique()

array(['6-10', '36-40', '11-15', '21-25', '0-5', '26-30', '16-20',
       '31-35'], dtype=object)

In [ ]:
cross_val_score(logistic_reg,pca_xtrain,y_train['grouped_classes'],cv=5)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([0.56156156, 0.49249249, 0.54654655, 0.50903614, 0.48795181])

In [ ]:
Predictions.to_html('LogisticRegression_predictions.html')

# Naive Bayes

In [ ]:
from sklearn.preprocessing import minmax_scale

In [ ]:
nb_train=minmax_scale(pca_xtrain,feature_range=(0,1))
nb_test=minmax_scale(pca_xtest,feature_range=(0,1))

In [ ]:
nb_class = MultinomialNB(alpha=0.01)
nb_class.fit(nb_train, y_train['grouped_classes'])

MultinomialNB(alpha=0.01)

In [ ]:
nb_class.predict(nb_test)

array(['0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '11-15', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '11-15', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '11-15', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5', '0-5',
       '0-5', 

In [ ]:
cross_val_score(nb_class,nb_train,y_train['grouped_classes'])

array([0.24324324, 0.24924925, 0.24924925, 0.25      , 0.2439759 ])